In [3]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os

In [4]:
display(os.getcwd())

'C:\\Users\\KIIT'

In [5]:
os.chdir("C:\\Users\\KIIT\\Desktop\\PRJ Movie Recommendation System\\")
display(os.getcwd())

'C:\\Users\\KIIT\\Desktop\\PRJ Movie Recommendation System'

In [6]:
data =pd.read_csv('movies.csv')

In [7]:
data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [8]:
display(data.shape)

(4803, 24)

In [9]:
features = ['genres','keywords','tagline','cast','director']
print(features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [10]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

None

In [11]:
display(data.isna().sum())

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [14]:
display(data[features].head())

,genres,keywords,tagline,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [15]:
display(data[features].isna().sum())

genres       28
keywords    412
tagline     844
cast         43
director     30
dtype: int64

In [19]:
for i in features:
    data[features]=data[features].fillna('')
display(data.head())

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [22]:
combined = data['genres']+''+data['keywords']+''+['tagline']+''+data['cast']+''+data['director']
display(combined.head())

0    Action Adventure Fantasy Science Fictioncultur...
1    Adventure Fantasy Actionocean drug abuse exoti...
2    Action Adventure Crimespy based on novel secre...
3    Action Crime Drama Thrillerdc comics crime fig...
4    Action Adventure Science Fictionbased on novel...
dtype: object

In [24]:
vector = TfidfVectorizer()
feature_vectors = vector.fit_transform(combined)
display(feature_vectors.shape)
print(feature_vectors)

(4803, 24549)

  (0, 3131)	0.18368883861825752
  (0, 18506)	0.28782238960307566
  (0, 14880)	0.16736305697696868
  (0, 13009)	0.23508681813133422
  (0, 20774)	0.1742625175556606
  (0, 23649)	0.20640081302774052
  (0, 20093)	0.22649007183030365
  (0, 19231)	0.22649007183030365
  (0, 24513)	0.20925186720922898
  (0, 24193)	0.24677379086445017
  (0, 20363)	0.28782238960307566
  (0, 4109)	0.2577987925408172
  (0, 23466)	0.14149042180361437
  (0, 20476)	0.3634047696119983
  (0, 9077)	0.19801478419149682
  (0, 3916)	0.23196384161871197
  (0, 8274)	0.28782238960307566
  (0, 19553)	0.1048094878039434
  (0, 7943)	0.12430361555048966
  (0, 319)	0.09682851786217786
  (0, 135)	0.08268679818774284
  (1, 23176)	0.21582552867988472
  (1, 8095)	0.2562698802890311
  (1, 24392)	0.20399631415121874
  (1, 3789)	0.20931544214864117
  :	:
  (4801, 11150)	0.3391068354439613
  (4801, 19970)	0.3391068354439613
  (4801, 19066)	0.3391068354439613
  (4801, 4808)	0.3391068354439613
  (4801, 21264)	0.3391068354439613
  (4801, 244

In [25]:
similar = cosine_similarity(feature_vectors)
print(similar)

[[1.         0.02210548 0.01738357 ... 0.         0.         0.        ]
 [0.02210548 1.         0.00895073 ... 0.01812976 0.         0.        ]
 [0.01738357 0.00895073 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.01812976 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [26]:
display(similar.shape)

(4803, 4803)

In [124]:
mymovie = input("Enter your favourite movie: ")

Enter your favourite movie: Iron Man


In [120]:
list_movieName = data['title'].tolist()
print(list_movieName)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [89]:
len(list_movieName)

4803

In [125]:
closeMatch = difflib.get_close_matches(mymovie,list_movieName)
print(closeMatch)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [126]:
close = closeMatch[0]
print(close)

Iron Man


In [127]:
indexMovie = data[data.title == close]['index'].values[0]
print(indexMovie)

68


In [128]:
similarity_score = list(enumerate(similar[indexMovie]))
print(similarity_score)


[(0, 0.017579752590085273), (1, 0.04238699414290289), (2, 0.007231100546285054), (3, 0.007479008622117209), (4, 0.018130273379334715), (5, 0.006883860224461705), (6, 0.036758083705716975), (7, 0.1519540596529891), (8, 0.0), (9, 0.04123879282370151), (10, 0.018956842751927782), (11, 0.006802157363559876), (12, 0.0), (13, 0.006700550921230815), (14, 0.05521319441200348), (15, 0.0), (16, 0.20511240272278192), (17, 0.0069493599836114965), (18, 0.01696744282026964), (19, 0.006783213809776819), (20, 0.08069982088245155), (21, 0.0067088276069672175), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.169635871807371), (27, 0.016363048765753455), (28, 0.06648807173844111), (29, 0.007187154148618755), (30, 0.08267507735002222), (31, 0.25087406725460676), (32, 0.0), (33, 0.14423714101502566), (34, 0.0), (35, 0.03698046203888804), (36, 0.03510159242472855), (37, 0.0), (38, 0.11870670391353719), (39, 0.09220108925922015), (40, 0.0), (41, 0.018398682808085133), (42, 0.0), (43, 0.03444947237581047),

In [129]:
len(similarity_score)

4803

In [130]:
sortedSimilar = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sortedSimilar)


[(68, 1.0), (79, 0.2914511571548304), (31, 0.25087406725460676), (16, 0.20511240272278192), (26, 0.169635871807371), (7, 0.1519540596529891), (203, 0.1486571729317591), (33, 0.14423714101502566), (3623, 0.13604748908527223), (511, 0.13214093991800427), (174, 0.1320354459113128), (46, 0.13058895932150177), (101, 0.12408920593606274), (64, 0.1231439436778933), (38, 0.11870670391353719), (307, 0.11199538932426202), (788, 0.11197375343038556), (1406, 0.1088293866220015), (3207, 0.10427620693833356), (3166, 0.10404390784814296), (85, 0.1033755608522754), (2186, 0.10317902389818451), (182, 0.10195846182120155), (2235, 0.10126569607757398), (2221, 0.09882231484033115), (1282, 0.09839876440139306), (1553, 0.09521541699482838), (3172, 0.09472697741559244), (4241, 0.09243660584834995), (39, 0.09220108925922015), (3116, 0.09199840745830998), (783, 0.0917376519058692), (2648, 0.09129730989750848), (4033, 0.08945886279621841), (1311, 0.0894263942548904), (3321, 0.08530762131116273), (1664, 0.084836

In [131]:
print("Suggestions for you ! : \n")
i=1
for k in sortedSimilar:
    index=k[0]
    titleIndex=data[data.index==index]['title'].values[0]
    if(i<30):
        print(i,'',titleIndex)
        i+=1

Suggestions for you ! : 

1  Iron Man
2  Iron Man 2
3  Iron Man 3
4  The Avengers
5  Captain America: Civil War
6  Avengers: Age of Ultron
7  X2
8  X-Men: The Last Stand
9  Made
10  X-Men
11  The Incredible Hulk
12  X-Men: Days of Future Past
13  X-Men: First Class
14  X-Men: Apocalypse
15  The Amazing Spider-Man 2
16  The Expendables 3
17  Deadpool
18  The Nativity Story
19  Awake
20  The Best Man
21  Captain America: The Winter Soldier
22  The Kite Runner
23  Ant-Man
24  A Scanner Darkly
25  The Man
26  The Hunting Party
27  Se7en
28  The Contender
29  The Salon
